In [1]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import platform
import shutil
import os.path
import fileinput
import ipywidgets as widgets
from io import BytesIO
import xml.etree.cElementTree as ET
from functools import partial
import time
import getpass
from encrypt import decryptCredentials,decryptString
import gmaps
import psycopg2 as db
from psycopg2.extensions import AsIs
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from urllib.parse import urljoin
from IPython.display import clear_output, display

options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Windows'):
    options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd,verbose=True,overwrite=False):
    with Popen(cmd.split(),**options) as p:
        if verbose and not overwrite:
            for line in p.stdout:
                print(line, end='')
        if verbose and overwrite:
            for line in p.stdout:
                clear_output(wait=True)
                display(line)
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)


def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)

def replace(file,pattern,replace):
    fileinput.close()
    for line in fileinput.input(file, inplace=True):
        print( re.sub(pattern,
                      replace,
                      line.rstrip()
                      ) 
             )
        
if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())   

# Getting data from metno thredds servers
## Creating a virtual machine with the fimex utility

In [3]:
instanceName = 'fimex-kau'
username = 'jose-luis'

createInstance = '''
gcloud compute instances create {} 
--zone europe-west3-a 
--image-family ubuntu-1604-lts 
--image-project ubuntu-os-cloud 
--machine-type n1-highmem-16 
--scopes default,storage-rw
--boot-disk-size 200GB
'''
#--boot-disk-size 200GB \
#--machine-type n1-highcpu-32  n1-standard-2\
deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''

if (platform.system() == 'Linux'):
    keyDir = ('/home/jose-luis/.ssh/fimexKeys')

if (platform.system() == 'Windows'):
    keyDir = ('c:/Users/jose_luis_guerrero/fimexKeys')
    

ip=''
instanceExists,ip = isInstance(instanceName)

if instanceExists:
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    if os.path.exists(keyDir):
        shutil.rmtree(keyDir)
    os.mkdir(keyDir)
    generateSSHKey(username,keyDir)
    keyFile = os.path.join(keyDir,username + '.pub')
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    #callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    replace(keyFile,"^{}:".format(username),"")
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

Instance fimex-kau is 35.242.200.24
The ip of fimex-kau is 35.242.200.24


## Updating fabfile.py with credentials and ip

In [4]:
if (platform.system() == "Linux"):
    callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
    callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
    callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
    callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('ncquery',ip))

fabfile = os.path.join("C:\\Users\\jose_luis_guerrero\\Envs\\mylai\\prognos_calibration","fabfile.py")
    
if (platform.system() == "Windows"):    
    replace(fabfile, "^env\.hosts.*",         "env.hosts=['{}']".format(ip))
    replace(fabfile, "^env\.user.*",          "env.user='{}'".format(username))
    replace(fabfile, "^env\.key_filename.*",  "env.key_filename='{}'".format(os.path.join(keyDir,username)))
    replace(fabfile, "^env\.roledefs.*",      "env.roledefs={{'{}':['{}'],".format('ncquery',ip))

#Testing connection
#Adding key to remote machine
time.sleep(2) #Giving time for the editing to work
if not instanceExists:
    p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
    print(p.communicate())

time.sleep(2)

## Setting fimex in the remote machine

In [4]:
if not instanceExists:
    callPopen('fab installDependencies')

## Querying extent of the basin
### Connecting to geodatabase

In [6]:
token = b'gAAAAABayyyn8ZnEstm8ZQqClUYQ-IqFFuMO4QTbmFJADHWBAcirh52s5stDwSwtVK7qVm5tzdTNFxTQjuRF28b1t2rosFSl_nnTowWrD4itOjkzF7s6Kg_qa1Adqpj59OAfBapgkToUQUHvEFY1Njc4he36AC76gmb8t0CJCq4ze2pDHWIlGdDacZxQ1jq14uLVxrFfCTSxDPX8Mx9W1av723etkOdWvw=='
key = getpass.getpass('Password: ')
credentials = decryptCredentials(token,key)

# Testing connection to database
try : 
    conn = db.connect("dbname={} user={} host={} password={}".format(credentials['database'],credentials['username'],credentials['host'],credentials['password']))
    print('Connection successful!')
    conn.close()
except :
    print("Unable to connect")
    


Password: ········
Connection successful!


## Getting list of openDAP links with high-resolution (1km2) temperature and precipitation data

In [8]:
getList=False
baseURL='https://thredds.met.no/thredds/catalog/ngcd/version_18.09/catalog.html'
maxTemp='TX'
minTemp='TN'
model='type1'

only_a_tags = SoupStrainer("a", href=True)

def getSoup(url,re_str):
    request=requests.get(url)
    soup=BeautifulSoup(request.content,'lxml',parse_only=only_a_tags)
    link_soup=soup.find_all('a',text=re.compile(re_str))
    links=[]
    for i in link_soup:
        links.append(urljoin(url,i['href']))
    return links

def getFileList(var):
    allLinks=[]
    for var_link in getSoup(baseURL,var):
        for model_link in getSoup(var_link,model):
            for year_link in getSoup(model_link,'^[0-9]{4}/$'):
                print('Processing {}'.format(year_link))
                for month_link in getSoup(year_link,'^[0-9]{2}/$'):
                    for day_link in getSoup(month_link,'\\.nc$'):
                        for opendap_link in getSoup(day_link,'^/thredds/dodsC/'):
                            allLinks.append(re.sub(r'\.html$', '', opendap_link))
    return allLinks
    

if getList:
    maxTempLinks=getFileList(maxTemp)
    minTempLinks=getFileList(minTemp)
    with open('min_temp_nc_files_{}.txt'.format(model), 'w') as f:
        for item in minTempLinks:
            f.write("%s\n" % item)
    with open('max_temp_nc_files_{}.txt'.format(model), 'w') as f:
        for item in maxTempLinks:
            f.write("%s\n" % item)    

These list of opendap links to netcdf files are then passed to a virtual machine containing the fimex utility in order to get data only for the basin of interest.

## Creating data for basins
They should have been created beforehand and are queried from the geodatabase

In [13]:
#box='BOX(10.680739923014 59.3434714764489,11.1305912803339 59.859034461037)' #Vanjsø
getBox=True
if getBox:
    basinSchema='francois'
    conn = db.connect("dbname={} user={} host={} password={}".format(credentials['database'],
                                                                           credentials['username'],
                                                                            credentials['host'],
                                                                            credentials['password']
                                                                          )
                           )
    #And a test query
    cursor = conn.cursor()
    
    cursor.execute('''SELECT a.station_name,
                      Box2D(ST_Transform(St_Buffer(St_Envelope(a.basin),2000),4326))
                      FROM %(tableSHP)s AS a
                      WHERE a.station_id=108;
                   '''
                      ,{"tableSHP": AsIs(basinSchema+'.resultsShp')}
                  )
    conn.commit()
    rows=cursor.fetchall()

    for row in rows:
        name = 'Langtjern' #row[0]
        box = row[1].replace(',',' ').replace(' ','\\\,')
        print('Processing {}'.format(name))
        print('\n\n\n')
       
        callPopen('''fab getDataForBasin:{},{},{},{},{} '''.format('rain_nc_files_{}.txt'.format(model),
                                                                       box,
                                                                       './rain_{}_nc'.format(name),
                                                                       'rain_{}.nc'.format(name),
                                                                        'RR'), verbose=True, overwrite=True
        )
        callPopen('''fab getDataForBasin:{},{},{},{},{} '''.format('temperature_nc_files_{}.txt'.format(model),
                                                                       box,
                                                                       './temperature_{}_nc'.format(name),
                                                                       'temperature_{}.nc'.format(name),
                                                                       'TG'),verbose=True,overwrite=True
        ) 
        callPopen('''fab getDataForBasin:{},{},{},{},{} '''.format('min_temp_nc_files_{}.txt'.format(model),
                                                                       box,
                                                                       './min_temp_{}_nc'.format(name),
                                                                       'min_temp_{}.nc'.format(name),
                                                                       'TN'),verbose=True,overwrite=True
        ) 
        callPopen('''fab getDataForBasin:{},{},{},{},{} '''.format('max_temp_nc_files_{}.txt'.format(model),
                                                                       box,
                                                                       './max_temp_{}_nc'.format(name),
                                                                       'max_temp_{}.nc'.format(name),
                                                                       'TX'),verbose=True,overwrite=True
        ) 
    conn.close()


'Disconnecting from 35.242.200.24... done.\n'